In [12]:
import pandas as pd
import numpy as np
import datetime
from haversine import haversine
import os
import json


zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [13]:
dict_SOTF={"4687":{"Market":"BAY SHORE, NY","Date":"2019-02-08"},
           
           "5378":{"Market":"MISHAWAKA, IN","Date":"2019-03-01"},
           "4555":{"Market":"NORTH HIGHLANDS, CA","Date":"2019-03-01"},
           "1918":{"Market":"SACRAMENTO, CA","Date":"2019-03-01"},
           "5370":{"Market":"SARASOTA, FL","Date":"2019-03-01"},
           
           "4674":{"Market":"CITRUS HEIGHTS, CA","Date":"2019-03-29"},
           "4685":{"Market":"LONGMONT, CO","Date":"2019-03-29"},
           "5390":{"Market":"NILES, OH","Date":"2019-03-29"},
           "4693":{"Market":"VANCOUVER, WA","Date":"2019-03-29"},
           
           "4690":{"Market":"HUNTINGTON, WV","Date":"2019-04-26"},
           "5383":{"Market":"MOUNT PLEASANT, PA","Date":"2019-04-26"},
          }

In [14]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")
store_list.head(2)
store_list_SOTF=store_list[store_list['location_id'].isin(dict_SOTF.keys())]

In [15]:
zips_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
zips_by_store[zips_by_store['revenue_flag'].isin(dict_SOTF.keys())]
# No P/S label found

,net_transaction_amt,transactions,zip,revenue_flag,transaction_flag,location_id


In [16]:
store_available_lat_lng=store_list_SOTF[['location_id','latitude_meas','longitude_meas']]
store_available_lat_lng['latitude_meas']=store_available_lat_lng['latitude_meas'].astype(float)
store_available_lat_lng['longitude_meas']=store_available_lat_lng['longitude_meas'].astype(float)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
store_available_lat_lng

,location_id,latitude_meas,longitude_meas
667,1918,38.563259,-121.380165
972,4555,38.691322,-121.381584


In [18]:
dict_SOTF.keys()

dict_keys(['4687', '5378', '4555', '1918', '5370', '4674', '4685', '5390', '4693', '4690', '5383'])

In [19]:
store_from_Dom_list=[x for x in list(dict_SOTF.keys()) if x not in store_available_lat_lng['location_id'].tolist()]
print(len(store_from_Dom_list))
print(store_from_Dom_list)

new_store_lat_lng_dict={"4687":[40.7397339,-73.2365623],
                   
                   "5378":[41.6826079,-86.187903],
                   # "4555":[38.6913996,-121.3816683], available
                   # "1918":[38.5630068,-121.3805318], available
                   "5370":[27.3404875,-82.4994362],
                   
                   "4674":[38.7211498,-121.292759],
                   "4685":[40.1973844,-105.1037418],
                   "5390":[41.2151883,-80.7497738],
                   # Lack of "4693"
                   
                   "4690":[38.409634,-82.3248814],
                   "5383":[40.1252467,-79.5510018],
                  } 

9
['4687', '5378', '5370', '4674', '4685', '5390', '4693', '4690', '5383']


In [20]:
SOTF_lat_lng_dict=new_store_lat_lng_dict.copy()
SOTF_lat_lng_dict.update({"1918":[38.563259,-121.380165],"4555":[38.691322,-121.381584]})

In [21]:
i_counter=0
sotf_zips=pd.DataFrame()
for SOTF in SOTF_lat_lng_dict.keys():
    store_center=SOTF_lat_lng_dict[SOTF]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist <= 10:
            df=pd.DataFrame({"location_id":SOTF,'zip_cd':zip_cd,"revenue_flag":"zips_in_10_miles","distance_miles":dist},index=[i_counter])
            i_counter+=1
            sotf_zips=sotf_zips.append(df)

In [22]:
sotf_zips=sotf_zips[['location_id','zip_cd','revenue_flag','distance_miles']]

In [27]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Q1_SOTF_Zips'

In [28]:
sotf_zips.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Q1_SOTF_Zips/BL_zips_for_SOTF_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
